In [2]:
import numpy as np
import pandas as pd
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import yaml
#from models import *

In [45]:
def graph_model(out, N, p):
    S, I, A, R = out
    fig = plt.figure(facecolor='w', figsize=[2*6.4, 2*4.8])
    ax = fig.add_subplot(111, facecolor='#dddddd', axisbelow=True)
    ax.plot(t, S, 'b', alpha=0.5, lw=2, label='Susceptible')
    ax.plot(t, I, 'r', alpha=0.5, lw=2, label='Symptomatic')
    ax.plot(t, A, 'k', alpha=0.5, lw=2, label='Asymptomatic')
    ax.plot(t, R, 'g', alpha=0.5, lw=2, label='Recovered')

    ax.set_xlabel('Time (days)')
    ax.set_ylabel('Cases')
    ax.set_xlim(0,500)
    ax.yaxis.set_tick_params(length=0)
    ax.xaxis.set_tick_params(length=0)
    ax.grid(b=True, which='major', c='w', lw=2, ls='-')
    legend = ax.legend()
    plt.title("Vibrio Cholerae concentration = " + str(p['W']) + " cells/mL")
    legend.get_frame().set_alpha(0.5)
    for spine in ('top', 'right', 'bottom', 'left'):
        ax.spines[spine].set_visible(False)
    plt.show()
    
def model_dengue(y, t, N, p):

    Sh, Eh, Iha, Ihs, Rh, Sv, Ev, Iv = y
    
    
    lambda_h = p['b_h'] * p['beta_h'] * Iv / p['N_v']
    lambda_v = p['b_v'] * p['beta_v'] * (Iha + Ihs) / p['N_h']
    
    dSh = -lambda_h * Sh
    dEh = -lambda_h * Sh - p['nu_h'] * Eh
    dIha = p['psi'] * p['nu_h'] * Eh - p['gamma_h'] * Iha
    dIhs = (1 - p['psi']) * p['nu_h'] * Eh - p['gamma_h'] * Ihs
    dRh = p['gamma_h'] * (Iha + Ihs)
    dSv = -lambda_v * Sh
    dEv = -lambda_v * Sh - p['nu_v'] * Ev
    dIv = p['nu_v'] * Ev - p['mu_v'] * Iv

#     w = p['a'] * p['W'] / (p['kappa'] + p['W'])

#     dS = -(p['beta'] * (I + A) + w) * S    
#     dI = (1 - p['p']) * S * (p['beta']  * (I + A) + w) - p['gamma'] * I
#     dA = p['p'] * S * (p['beta'] * (I + A) + w) - p['gamma'] * A
#     dR = p['gamma'] * (I + A)

    return dSh, dEh, dIha, dIhs, dRh, dSv, dEv, dIv

In [56]:
with open('config.yaml', 'r') as in_file:
    params = yaml.load(in_file, Loader=yaml.FullLoader)['DENGUE']
p = params['PARAMETERS']
s_init = params["INITIAL_STATES"]

{'Sh': 8, 'Eh': 0, 'Iha': 1, 'Ihs': 0, 'Rh': 0, 'Sv': 0, 'Ev': 0, 'Iv': 0}

In [58]:
# s_init = {'Sh': 10 - 2,\
#           'Eh': 0,\
#           'Iha': 1,\
#           'Ihs': 0,\
#           'Rh': 0,\
#           'Sv': 0,\
#           'Ev': 0,\
#           'Iv' : 0}

# p = {'psi' : 0.9,\
#      'nu_h': 2e-8,\
#      'nu_v': int(1e5),\
#      'gamma_h': 1e6,\
#      'mu_v': 0.2,\
#      'b_h': 2,\
#      'b_v': 2,\
#      'beta_h': 1/7,\
#      'beta_v': 1/7,\
#      'N_h': 100,\
#      'N_v': 100}

# tt = {'t2': 22,\
#         'tt': 33,\
#         'ttt': 44}

out = {'DENGUE': {"PARAMETERS": p, "INITIAL_STATES": s_init}, 'WNV': tt}

days = 3000
t = np.linspace(0, days, days*500)

# -----------------
s = 0 #PLACEHOLDER.. WHY IS THIS NEEDED?-----------------------------------
# -----------------
y0 = s_init['Sh'], s_init['Eh'], s_init['Iha'], s_init['Ihs'],\
    s_init['Rh'], s_init['Sv'], s_init['Ev'], s_init['Iv']
output = odeint(model_dengue, y0, t, args = (s,p))
Sh, Eh, Iha, Ihs, Rh, Sv, Ev, Iv = output.T
#graph_model(output.T, N, p)

In [53]:
# with open('config.yaml', 'w') as out_file:
#     yaml.dump(out, out_file, sort_keys=False)